## Read Labels

In [1]:
!ls -l tuh/train/normal/ | wc -l

    1372


In [2]:
!ls -l tuh/train/abnormal/ | wc -l

    1347


In [3]:
!ls -l tuh/eval/abnormal/ | wc -l

     127


In [4]:
!ls -l tuh/eval/normal/ | wc -l

     151


In [5]:
!echo $CONDA_DEFAULT_ENV

xai


In [6]:
# read sample data file
import numpy as np

test_file = np.load("tuh/train/normal/10005_0_65_F_1133.npy")
test_file.shape

(118, 19, 500)

In [7]:
%%capture
import numpy as np
from tqdm import tqdm
import glob

import scipy.io

import os
import mne
import matplotlib.pyplot as plt

In [32]:
test_abnormal_files = glob.glob('./tuh/eval/abnormal/*.npy')
test_normal_files = glob.glob('./tuh/eval/normal/*.npy')
train_normal_files = glob.glob('./tuh/train/normal/*.npy')
train_abnormal_files = glob.glob('./tuh/train/abnormal/*.npy')

In [33]:
len(train_abnormal_files), len(test_normal_files)

(1346, 150)

In [34]:
import random
#val_normal_files = random.sample(train_normal_files, 100)
#val_abnormal_files = random.sample(train_abnormal_files, 100)
train_normal_files = random.sample(train_normal_files, 300)
train_abnormal_files = random.sample(train_abnormal_files, 300)

In [35]:
len(train_abnormal_files), len(train_normal_files)

(300, 300)

In [36]:
%%capture

train_normal_features=[np.load(f) for f in train_normal_files]
train_abnormal_features=[np.load(f) for f in train_abnormal_files]
test_normal_features=[np.load(f) for f in test_normal_files]
test_abnormal_features=[np.load(f) for f in test_abnormal_files]

In [37]:
train_normal_features[0].shape

(118, 19, 500)

In [38]:
test_abnormal_features[0].shape

(118, 19, 500)

In [39]:
train_normal_labels=[len(i)*[0] for i in train_normal_features]
train_abnormal_labels=[len(i)*[1] for i in train_abnormal_features]

test_normal_labels=[len(i)*[0] for i in test_normal_features]
test_abnormal_labels=[len(i)*[1] for i in test_abnormal_features]

In [40]:
train_features = train_normal_features + train_abnormal_features
train_labels = train_normal_labels + train_abnormal_labels

test_features = test_normal_features + test_abnormal_features
test_labels = test_normal_labels + test_abnormal_labels

In [41]:
del train_normal_features
del train_abnormal_features

In [42]:
train_features = np.vstack(train_features)
train_labels = np.hstack(train_labels)

In [43]:
test_features = np.vstack(test_features)
test_labels = np.hstack(test_labels)

In [44]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GroupKFold
gkf=GroupKFold()
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import StandardScaler
#https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
class StandardScaler3D(BaseEstimator,TransformerMixin):
    #batch, sequence, channels
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self,X,y=None):
        self.scaler.fit(X.reshape(-1, X.shape[2]))
        return self

    def transform(self,X):
        return self.scaler.transform(X.reshape( -1,X.shape[2])).reshape(X.shape)

In [45]:
scaler = StandardScaler3D()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

## ChronoNet

In [22]:
from tqdm import tqdm
import torch.nn as nn
import torch
from torch.autograd import Variable

class Block(nn.Module):
  def __init__(self,inplace):
    super().__init__()
    self.conv1=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=2,stride=2,padding=0)
    self.conv2=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=4,stride=2,padding=1)
    self.conv3=nn.Conv1d(in_channels=inplace,out_channels=32,kernel_size=8,stride=2,padding=3)
    self.relu=nn.ReLU()

  def forward(self,x):
    x1=self.relu(self.conv1(x))
    x2=self.relu(self.conv2(x))
    x3=self.relu(self.conv3(x))
    x=torch.cat([x1,x3,x3],dim=1)
    return x

class ChronoNet(nn.Module):
  def __init__(self,channel):
    super().__init__()
    self.block1=Block(channel)
    self.block2=Block(96)
    self.block3=Block(96)
    self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
    self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
    self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
    self.gru_linear=nn.Linear(62,1)
    self.flatten=nn.Flatten()
    self.fc1=nn.Linear(32,1)
    self.relu=nn.ReLU()

  def forward(self,x):
    x = x.squeeze()
    x=self.block1(x)
    x=self.block2(x)
    x=self.block3(x)
    x=x.permute(0,2,1)
    gru_out1,_=self.gru1(x)
    gru_out2,_=self.gru2(gru_out1)
    gru_out=torch.cat([gru_out1,gru_out2],dim=2)
    gru_out3,_=self.gru3(gru_out)
    gru_out=torch.cat([gru_out1,gru_out2,gru_out3],dim=2)
    linear_out=self.relu(self.gru_linear(gru_out.permute(0,2,1)))
    gru_out4,_=self.gru4(linear_out.permute(0,2,1))
    x=self.flatten(gru_out4)
    x=self.fc1(x)
    out = torch.sigmoid(x)
    return out

/Users/mohamedr/opt/anaconda3/envs/xai/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:

#device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
device = torch.device("cpu")

train_features = torch.Tensor(train_features).float().to(device)
train_labels = torch.Tensor(train_labels).float().to(device)
batch_size = 128
train_data = torch.utils.data.TensorDataset(train_features, train_labels)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)

test_features = torch.Tensor(test_features).float().to(device)
test_labels = torch.Tensor(test_labels).float().to(device)

test_data = torch.utils.data.TensorDataset(test_features, val_labels)
test_iter = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)

In [28]:
def evaluate_model(model, loss_func, data_iter):
    model.eval()
    loss_sum, n = 0, 0
    with torch.no_grad():
        for x, y in data_iter:
            y_pred = model(x)
            y_pred = y_pred.squeeze()
            loss = loss_func(y_pred,y)
            loss_sum += loss.item()
            n += 1
        return loss_sum / n

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix

def print_accuracy(model, labels, features):
    with torch.no_grad():
        y_hat = model(features)

    yhat = [0 if i<0.5 else 1 for i in y_hat]
    ytrue = labels.numpy()
    ypreds = yhat

    print("Accuracy: ", accuracy_score(ytrue, ypreds))
    print("Confusion Matrix: ")
    print(confusion_matrix(ytrue, ypreds))

In [47]:
n_chans = 19
model=ChronoNet(n_chans)
model.to(device)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for epoch in range(1, epochs + 1):
    print("epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train")
    print("#####")
    print("loss:", loss_sum / (t+1))
    print(print_accuracy(model, train_labels, train_features))
    print("Val")
    print("#####")
    print("loss:", val_loss)
    print(print_accuracy(model, val_labels, val_features))

epoch 1


100%|█████████████████████████████████████████| 554/554 [03:57<00:00,  2.33it/s]


Train
#####
Train loss: 0.6344686868173551
Accuracy:  0.7718644067796611
Confusion Matrix: 
[[29822  5578]
 [10574 24826]]
None
Val
#####
Val loss: 0.6312071293008094
Accuracy:  0.7574613117170228
Confusion Matrix: 
[[14868  2832]
 [ 5067  9801]]
None
epoch 2


100%|█████████████████████████████████████████| 554/554 [04:29<00:00,  2.05it/s]


Train
#####
Train loss: 0.6003486813190612
Accuracy:  0.7966949152542373
Confusion Matrix: 
[[31722  3678]
 [10716 24684]]
None
Val
#####
Val loss: 0.6185130250220205
Accuracy:  0.7772660280029476
Confusion Matrix: 
[[15631  2069]
 [ 5185  9683]]
None
epoch 3


...

Train
#####
Train loss: 0.5664093874206612
Accuracy:  0.8675423728813559
Confusion Matrix: 
[[32957  2443]
 [ 6935 28465]]
None
Val
#####
Val loss: 0.612890966499553
Accuracy:  0.7984831736674036
Confusion Matrix: 
[[15343  2357]
 [ 4206 10662]]
None
epoch 9


100%|█████████████████████████████████████████| 554/554 [04:07<00:00,  2.24it/s]


Train
#####
Train loss: 0.5646616778864327
Accuracy:  0.8697033898305084
Confusion Matrix: 
[[31489  3911]
 [ 5314 30086]]
None
Val
#####
Val loss: 0.6220441383474014
Accuracy:  0.7937239007614837
Confusion Matrix: 
[[14361  3339]
 [ 3379 11489]]
None
epoch 10


100%|█████████████████████████████████████████| 554/554 [03:55<00:00,  2.35it/s]


Train
#####
Train loss: 0.5657099489700923
Accuracy:  0.8672457627118644
Confusion Matrix: 
[[33598  1802]
 [ 7597 27803]]
None
Val
#####
Val loss: 0.6082649340816573
Accuracy:  0.8011545074920167
Confusion Matrix: 
[[15823  1877]
 [ 4599 10269]]
None


In [48]:
print_accuracy(model, val_labels, val_features)

Accuracy:  0.8011545074920167
Confusion Matrix: 
[[15823  1877]
 [ 4599 10269]]


In [ ]:
pp

## LSTM

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
    
class LSTM(nn.Module):
    def __init__(self, output_size, input_size, hidden_size, num_layers, num_channels=19):
        super(LSTM, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_channels = num_channels
        self.LSTMs = []
        self.vars_h0 = []
        self.vars_c0 = []
        for _ in range(self.num_channels):
            h0 = Variable(torch.zeros(self.num_layers*2, 32, 
                                      self.hidden_size))
            c0 = Variable(torch.zeros(self.num_layers*2, 32, 
                                      self.hidden_size))
            self.vars_h0.append(h0)
            self.vars_c0.append(c0)
            
            lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, 
                            bidirectional=True)
            self.LSTMs.append(lstm)
        self.linear = nn.Linear(hidden_size*2, 8)
        self.out_linear = nn.Linear(8*num_channels, output_size)

    def forward(self, x):
        ch_out = [] 
        for ch_idx in range(self.num_channels):
            h0 = self.vars_h0[ch_idx]
            c0 = self.vars_c0[ch_idx]    
            x_ch = x[:, ch_idx, :]
            x_ch = x_ch.unsqueeze(1)
            lstm_out, (hn, cn) = self.LSTMs[ch_idx](x_ch, (h0, c0))
            out = hn.view(-1, self.hidden_size*2)
            out = self.linear(out)
            ch_out.append(out)
        ch_out = torch.cat(ch_out, dim=1)
        out = self.out_linear(ch_out)
        out = torch.sigmoid(out)
        return out

In [ ]:
n_chans = 19
model = LSTM(output_size=1, input_size=500, hidden_size=32, num_layers=1, num_channels=n_chans)
loss_func = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
epochs = 10

for epoch in range(1, epochs + 1):
    print("epoch", epoch) 
    loss_sum, n = 0.0, 0
    model.train()
    for t, (x, y) in enumerate(tqdm(train_iter)):
        if x.size(0) != 32:
            break
        y_pred = model(x)
        y_pred = y_pred.squeeze()
        y = y.float()
        loss = loss_func(y_pred, y)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
        optimizer.zero_grad()
    
    val_loss = evaluate_model(model, loss_func, val_iter)
    print("Train loss:", loss_sum / (t+1))
    print("Val loss:", val_loss)

In [ ]:
print_accuracy(model, val_labels)